In [ ]:
import pandas as pd
import pickle
import numpy as np
from datetime import timedelta
from datetime import datetime

Sitzungssummary

In [ ]:
sitzungen = pd.read_csv('../00_data/raw/sitzungssummary.csv')
names = ['ID','UserID','UserAttribut','AufgabenID','Version','TestID','Art','HA','Fehler','Altdatum','Datum','Saetze','Korrektur','isExperiment']
sitzungen.columns = names

# filter 
sitzungen = sitzungen.loc[sitzungen['UserAttribut'] == 'Schüler']
sitzungen = sitzungen.loc[sitzungen['Art'] == 'GK']
sitzungen = sitzungen.drop(columns=['Altdatum'])
sitzungen = sitzungen[(sitzungen['Datum'] > '2022-06-21') & (sitzungen['Datum'] < '2022-10-31')]
sitzungen = sitzungen.loc[sitzungen['isExperiment']==1]
sitzungen = sitzungen.rename(columns={"ID": "UebungsID"})

XML Saetze

In [ ]:
xml = pd.read_csv('../00_data/raw/xmlsaetze.csv')
names = ['ID','UserID','UebungsID','UserAttribut','Testart','Testposition','SatzID','Erstloesung','Schussel','KorrekturBenutzt','Datum','Erfolg','XMLstring','Loesungsnr']
xml.columns = names

# filter 
xml = xml.loc[xml['UserAttribut'] == 'Schüler']
xml = xml.loc[xml['Testart'] == 'GK']
xml = xml[(xml['Datum'] > '2022-06-21') & (xml['Datum'] < '2022-10-31')]

xml['Loesungsnr']=xml['Loesungsnr'].str.split()
xml['MehrfachFalsch'] = xml['Loesungsnr'].str.len()
xml['MehrfachFalsch'] = xml['MehrfachFalsch'].replace([np.nan],0)
xml['MehrfachFalsch'] = xml['MehrfachFalsch'].astype('int')
xml['MehrfachFalsch'] = xml['MehrfachFalsch'] - 1

Saetze

In [ ]:
saetze = pd.read_csv('../00_data/raw/saetze.csv')
names = ['satzID','AufgabenID','Art','Satznr','Versionsnr','Aufgabe','Loesung','Hilfe','Erklaerung','Schwierigkeit','Gesamtschwierigkeit','XMLstring']
saetze.columns = names

# filter 
saetze = saetze.loc[saetze['Art'] == 'GK']

Schueler

In [ ]:
schueler = pd.read_csv('../00_data/raw/schueler.csv')
names = ['ID','Name','Passwort_express','Email','Loginname','Passwort','SessionID','Lehrername','LehrerID','Klassenname','Geschlecht','Klassenstufe','Anmeldedatum','Anmeldeklassenstufe','Altklasse','gesperrt','Aufgaben','Altaufgaben','done','Info','interventiongroup']
schueler.columns = names
schueler = schueler[['ID','Geschlecht','Klassenstufe','interventiongroup']]
schueler = schueler.rename(columns={"ID": "UserID"})

# filter
schueler['interventiongroup'] = np.where(
    (schueler.interventiongroup == 6) |
    (schueler.interventiongroup == 2) |
    (schueler.interventiongroup == 3) |
    (schueler.interventiongroup == 5) |
    (schueler.interventiongroup == '0'),
    schueler.interventiongroup,
    np.where(schueler.interventiongroup == 6, '6',
    np.where(schueler.interventiongroup == 2, '2',
    np.where(schueler.interventiongroup == 3, '3',
    np.where(schueler.interventiongroup == 5, '5',
    schueler.interventiongroup))))
)

Predictions

In [ ]:
pred = pd.read_csv('../00_data/raw/predictions.csv')
names = ['ID','UebungsID','SatzID','UserID','interventiongroup','prediction','seqMode','modus','versionline','version','Datum']
pred.columns = names

# preprocessing datetime
pred['Datum'] = pd.to_datetime(pred['Datum'].astype(str)) + pd.DateOffset(hours=2)
pred = pred[(pred['Datum'] > '2022-06-21') & (pred['Datum'] < '2022-10-31')]
pred['Date'] = pd.to_datetime(pred['Datum']).dt.date
pred['Time'] = pd.to_datetime(pred['Datum']).dt.time
pred['Minute'] = pd.to_datetime(pred['Datum']).dt.minute

# preprocessing interventiongroup
pred['interventiongroup'] = np.where((pred.interventiongroup == 6),'6',pred.interventiongroup)
schueler['interventiongroup'] = np.where((schueler.interventiongroup == 2),'2',schueler.interventiongroup)
schueler['interventiongroup'] = np.where((schueler.interventiongroup == 3),'3',schueler.interventiongroup)
schueler['interventiongroup'] = np.where((schueler.interventiongroup == 5),'5',schueler.interventiongroup)
pred['interventiongroup'] = np.where((pred.interventiongroup == '0'),'control',pred.interventiongroup)


Filtering

In [ ]:
#only keep records with users who are in sitzungen (which is filtered by isExperiment)
userlist = sitzungen.UserID.unique()
xml = xml.loc[xml['UserID'].isin(userlist)]
schueler = schueler.loc[schueler['UserID'].isin(userlist)]
valid = ['m','w','d']
schueler = schueler.loc[schueler['Geschlecht'].isin(valid)]
valid = ['9', '8', '10', '7', '5', '12', '6', '11', '13']
schueler = schueler.loc[schueler['Klassenstufe'].isin(valid)]

In [ ]:
# filter xml by uebungsIDs
uebungslist = sitzungen.UebungsID.unique()
xml = xml.loc[xml['UebungsID'].isin(uebungslist)]

Merge & Save files

In [ ]:
sitzungen = pd.merge(sitzungen, schueler, on='UserID')

sitzungen.to_pickle('../00_data/cleaned/sitzungssummary.pkl')
pred.to_pickle('../00_data/cleaned/predictions.pkl')
saetze.to_pickle('../00_data/cleaned/saetze.pkl')
xml.to_pickle('../00_data/cleaned/xmlsaetze.pkl')